In [4]:
import requests
import pandas as pd
text=pd.read_excel("Input.xlsx")
print(text)

             URL_ID                                                URL
0   blackassign0001  https://insights.blackcoffer.com/rising-it-cit...
1   blackassign0002  https://insights.blackcoffer.com/rising-it-cit...
2   blackassign0003  https://insights.blackcoffer.com/internet-dema...
3   blackassign0004  https://insights.blackcoffer.com/rise-of-cyber...
4   blackassign0005  https://insights.blackcoffer.com/ott-platform-...
..              ...                                                ...
95  blackassign0096  https://insights.blackcoffer.com/what-is-the-r...
96  blackassign0097  https://insights.blackcoffer.com/impact-of-cov...
97  blackassign0098  https://insights.blackcoffer.com/contribution-...
98  blackassign0099  https://insights.blackcoffer.com/how-covid-19-...
99  blackassign0100  https://insights.blackcoffer.com/how-will-covi...

[100 rows x 2 columns]


PREPROCESSING EXPLORATORY DATA ANALYSIS

In [ ]:
from bs4 import BeautifulSoup
for i in text.URL:
    para=requests.get(i)
    soup = BeautifulSoup(para.content, 'html5lib') 
    table = soup.find('p')
    print(table)


In [ ]:
for i in text.URL:
    para=requests.get(i)
    soup = BeautifulSoup(para.text, 'html.parser')
    content_elements = soup.find_all('p')
    print(len(content_elements))
    for element in content_elements:
        print(element.text)

GETTING DATA FROM THE LINKS FROM INPUT.XLSX AND SAVING THEM IN THE RESPECTIVE FILES WITH THEIR URL_ID AS NAME

In [20]:

data = pd.read_excel("Input.xlsx")

for index, row in data.iterrows():
    urlid = row['URL_ID']
    url = row['URL']
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        content_elements = soup.find_all('p')
        text = ""
        for element in content_elements:
            text += element.get_text() + "\n"
        with open(f"{urlid}.txt", "w", encoding="utf-8") as file:
            file.write(text)
            
        print(f"Text extracted from {urlid} and saved to {urlid}.txt")
        
    else:
        print(f"Failed to fetch content from {urlid}")


Text extracted from blackassign0001 and saved to blackassign0001.txt
Text extracted from blackassign0002 and saved to blackassign0002.txt
Text extracted from blackassign0003 and saved to blackassign0003.txt
Text extracted from blackassign0004 and saved to blackassign0004.txt
Text extracted from blackassign0005 and saved to blackassign0005.txt
Text extracted from blackassign0006 and saved to blackassign0006.txt
Text extracted from blackassign0007 and saved to blackassign0007.txt
Text extracted from blackassign0008 and saved to blackassign0008.txt
Text extracted from blackassign0009 and saved to blackassign0009.txt
Text extracted from blackassign0010 and saved to blackassign0010.txt
Text extracted from blackassign0011 and saved to blackassign0011.txt
Text extracted from blackassign0012 and saved to blackassign0012.txt
Text extracted from blackassign0013 and saved to blackassign0013.txt
Text extracted from blackassign0014 and saved to blackassign0014.txt
Text extracted from blackassign001

AFTER STORING THE TEXT FILES IN TEXT FOLDER USING NLTK WORD_TOKENISE AND SENT_TOKENISE AND AACCORDING TO THE POSITIVE NEGATIVE SCORE CALCULATION POLARITY AVERAGE WORDS CALUCULATED ACCORDING TO THE TEXT ANALYSIS DOCS AND SAVED IT IN THE RESPECTIVE COLUMNS IN THE OUTPUT.XLSX FILE

In [24]:
import os
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string
import re


positive_words = set()
with open("positive-words.txt", "r") as file:
    positive_words = set(file.read().splitlines())

negative_words = set()
with open("negative-words.txt", "r") as file:
    negative_words = set(file.read().splitlines())

def calculate_derived_variables(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]

    positive_score = sum(1 for token in tokens if token in positive_words)
    negative_score = sum(1 for token in tokens if token in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)

    sentences = sent_tokenize(text)
    word_count = len(tokens)
    sentence_count = len(sentences)
    avg_sentence_length = word_count / sentence_count
    complex_words = [word for word in tokens if len(word) > 2] 
    percentage_complex_words = (len(complex_words) / word_count) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_words_per_sentence = word_count / sentence_count


    syllable_count = sum(len(re.findall(r'[aeiouy]+', word)) for word in tokens)


    personal_pronouns = sum(1 for token in tokens if token in ["i", "we", "my", "ours", "us"])


    avg_word_length = sum(len(word) for word in tokens) / word_count if word_count > 0 else 0

    return {
        'Positive Score': positive_score,
        'Negative Score': negative_score,
        'Polarity Score': polarity_score,
        'Subjectivity Score': subjectivity_score,
        'Avg Sentence Length': avg_sentence_length,
        'Percentage of Complex Words': percentage_complex_words,
        'FOG Index': fog_index,
        'Average Number of Words Per Sentence': avg_words_per_sentence,
        'Complex Word Count': len(complex_words),
        'Word Count': word_count,
        'Syllable Per Word': syllable_count / word_count if word_count > 0 else 0,
        'Personal Pronouns': personal_pronouns,
        'Avg Word Length': avg_word_length
    }

output_df = pd.DataFrame(columns=["URL_ID", "URL", "Positive Score", "Negative Score", "Polarity Score",
                                   "Subjectivity Score", "Avg Sentence Length", "Percentage of Complex Words",
                                   "FOG Index", "Average Number of Words Per Sentence", "Complex Word Count",
                                   "Word Count", "Syllable Per Word", "Personal Pronouns", "Avg Word Length"])

for filename in os.listdir("text"): 
    if filename.endswith(".txt"):
        with open(os.path.join("text", filename), "r", encoding="utf-8") as file:
            text = file.read()
        derived_variables = calculate_derived_variables(text)
        derived_variables["URL_ID"] = filename.split(".")[0]
        derived_variables["URL"] = "https://" + filename.split(".")[0] + ".com"
        output_df = output_df.append(derived_variables, ignore_index=True)
output_df.to_excel("output_results.xlsx", index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bunny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bunny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Bunny\AppData\Local\Temp\ipykernel_8040\3887679394.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(derived_variables, ignore_index=True)
C:\Users\Bunny\AppData\Local\Temp\ipykernel_8040\3887679394.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(derived_variables, ignore_index=True)
C:\Users\Bunny\AppData\Local\Temp\ipykernel_8040\3887679394.py:91: FutureWarning: The frame.append method is deprecated and will be removed from panda